In [3]:
# pip install ipython
# pip install pillow
# pip install TA_Lib-0.4.19-cp38-cp38-win_amd64.whl
# pip install xgboost
import numpy as np
import pandas as pd
import graphviz
import matplotlib.pyplot as plt

from IPython.display import Image

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

from PIL import Image as Image2

import tushare as ts
import datetime
from datetime import datetime as dtime
import mpl_finance as mpf
import seaborn as sns
from matplotlib.pylab import date2num
import talib

from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [4]:
crecard_df = pd.read_excel('信用评分卡模型.xlsx')
crecard_df.head()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73


In [5]:
'''
First, Multinomial Linear Regression model.
'''

lr_X = crecard_df.drop(columns='信用评分')
lr_Y = crecard_df['信用评分']

lr_model = LinearRegression()
lr_model.fit(lr_X,lr_Y)

print('各系数为:' + str(lr_model.coef_))
print('常数项系数k0为:' + str(lr_model.intercept_))

各系数为:[ 5.58658996e-04  1.62842002e-01  2.18430276e-01  6.69996665e-05
 -1.51063940e+00]
常数项系数k0为:67.16686603853253


In [6]:
# 用 R-squared 来查看回归模型拟合效果.

X2 = sm.add_constant(lr_X)
est = sm.OLS(lr_Y, X2).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   信用评分   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     337.6
Date:                Thu, 21 Jan 2021   Prob (F-statistic):          2.32e-211
Time:                        16:53:38   Log-Likelihood:                -2969.8
No. Observations:                1000   AIC:                             5952.
Df Residuals:                     994   BIC:                             5981.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         67.1669      1.121     59.906      0.000      64.967      69.367
月收入            0.0006   8.29e-05      6.735      0.000       0.000       0.001
年龄             0.1628      0.022      7.420      0.000       0.120       0.206
性别             0.2184      0.299      0.730      0.466      -0.369       0.806
历史授信额度        6.7e-05   7.78e-06      8.609      0.000    5.17e-05    8.23e-05
历史违约次数        -1.5106      0.140    -10.811      0.000      -1.785      -1.236
==============================================================================
Omnibus:                       13.180   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               12.534
Skew:                          -0.236   Prob(JB):                      0.00190
Kurtosis:                       2.721   Cond. No.                     4.27e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.27e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
'''
Next, GBDT model.
'''

gb_df = pd.read_excel('信用评分卡模型.xlsx')

gb_X = gb_df.drop(columns='信用评分')
gb_y = gb_df['信用评分']

gbX_train, gbX_test, gby_train, gby_test\
= train_test_split(gb_X, gb_y, test_size=0.2, random_state=123)

gb_model = GradientBoostingRegressor()  # 使用默认参数
gb_model.fit(gbX_train, gby_train)

gby_pred = gb_model.predict(gbX_test)
gb_vec = pd.DataFrame()
gb_vec['预测值'] = list(gby_pred)
gb_vec['实际值'] = list(gby_test)
gb_vec.head()

,预测值,实际值
0,70.776317,79
1,71.400321,80
2,73.734652,62
3,84.525339,89
4,71.091883,80


In [8]:
# 用 R-squared 来查看回归模型拟合效果.

gb_r2 = r2_score(gby_test, gb_model.predict(gbX_test))
print(gb_r2)

0.6754961914131579


In [9]:
'''
GBDT 的R2 相较于线性回归模型有改善,

现在来看XGBoost模型.
'''

xg_model = XGBRegressor()
xg_model.fit(gbX_train, gby_train)

xgy_pred = xg_model.predict(gbX_test)

xg_vec = pd.DataFrame()  # 创建一个空DataFrame 
xg_vec['预测值'] = list(xgy_pred)
xg_vec['实际值'] = list(gby_test)
xg_vec.head()

,预测值,实际值
0,74.623062,79
1,69.014954,80
2,76.393486,62
3,83.889977,89
4,71.568298,80


In [10]:
xg_r2 = r2_score(gby_test, xgy_pred)
print(xg_r2)

0.5715437436791975


In [11]:
# parameters optimization

hyper = {'max_depth': [1, 3, 5], 'n_estimators': [30,50,70], 'learning_rate': [0.05, 0.1, 0.2]}  # 指定模型中参数的范围
opt_model = XGBRegressor()  # 构建回归模型
grid_search = GridSearchCV(opt_model, hyper, scoring='r2', cv=5)

grid_search.fit(gbX_train, gby_train)
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

In [12]:
better_model = XGBRegressor(max_depth=3, n_estimators=50, learning_rate=0.1)
better_model.fit(gbX_train, gby_train)

better_xg_r2 = r2_score(gby_test, better_model.predict(gbX_test))
print(better_xg_r2)

0.6884486054771359


In [13]:
'''
调优后的 R-squared 值是三种模型中最好的.

另外树模型有个优点: 很多数据预处理可以省略,如(归一、标准化、缺失、共线())
'''



'\n调优后的 R-squared 值是三种模型中最好的.\n\n另外树模型有个优点: 很多数据预处理可以省略,如(归一、标准化、缺失、共线())\n'